In [22]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adtk.detector import AutoregressionAD
from adtk.data import validate_series

In [23]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["dadosVivamente"]
collection = db["postsAgregadoComRespostas"]

In [24]:
# Filtrando documentos com o atributo maior ou igual a 2
filtro = {"suicida": {"$eq": "2"}}
documentos = collection.find(filtro)

In [25]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))

In [26]:
# Visualizando o DataFrame
df.head()

,_id,idade,sexo,id_usuario,nivel,pessimismo,tristeza,fracasso,prazer,culpa,...,quantAmigos,postMessage,postCreated_time,diaDaSemana,hora,minutos,diaDoMes,mes,ano,postStory
0,59305142c46708001d041a16,47,M,1402702123120502,2.0,1,3,3,3,1,...,491,Beleza aí tio quando vai vir nas jantas do tir...,2017-05-30 03:16:28,Terça-feira,3,16,30,5,2017,NaN
1,59305142c46708001d041a17,47,M,1402702123120502,2.0,1,3,3,3,1,...,491,EXCLUSIVO PARA VIDEIRA E REGIÃO! \n2ª FEIRA - ...,2017-05-28 15:42:56,Domingo,15,42,28,5,2017,Ricardo Luiz Cunha with Clarice Cunha and 37 o...
2,59305142c46708001d041a18,47,M,1402702123120502,2.0,1,3,3,3,1,...,491,NaN,2017-05-13 13:00:00,Sábado,13,0,13,5,2017,Ricardo Luiz Cunha shared Sergio Mallandro Ofi...
3,59305142c46708001d041a19,47,M,1402702123120502,2.0,1,3,3,3,1,...,491,NaN,2017-04-20 20:59:08,Quinta-feira,20,59,20,4,2017,Ricardo Luiz Cunha shared O que existe dentro...
4,59305142c46708001d041a1a,47,M,1402702123120502,2.0,1,3,3,3,1,...,491,Ricardo Luiz Cunha pra você!,2017-04-05 22:31:20,Quarta-feira,22,31,5,4,2017,Clarice Cunha shared Boa Música's video.


In [27]:
df['quantAddPhotoWithOthers'] = df['postStory'].apply(
    lambda x: 1 if isinstance(x, str) and (
            'added a new photo — with' in x.lower() or
            'new photos — with' in x.lower() or
            ('photo to the album:' in x.lower() and 'with' in x.lower() and 'and' in x.lower()) or
            'others' in x.lower()
    ) else 0
)

df['quantAddPhotoWith'] = df['postStory'].apply(
    lambda x: 1 if isinstance(x, str) and (
            'added a new photo — with' in x.lower() or
            'new photos — with' in x.lower() or
            ('photo to the album:' in x.lower() and 'with' in x.lower())
    ) else 0
)

df['quantAddPhoto'] = df['postStory'].apply(
    lambda x: (x.lower().count('added a new photo') +
               x.lower().count('new photos') +
               x.lower().count('photo to the album:')) if isinstance(x, str) else 0
)

df['quantSharedPhoto'] = df['postStory'].apply(
    lambda x: (x.lower().count('shared ') if 'photo' in x.lower() else 0) if isinstance(x, str) else 0
)

df['quantSharedVideo'] = df['postStory'].apply(
    lambda x: (x.lower().count('shared ') if 'video' in x.lower() else 0) if isinstance(x, str) else 0
)

df['quantSharedLink'] = df['postStory'].apply(
    lambda x: x.lower().count('shared a link') if isinstance(x, str) else 0
)

df['quantSharedPost'] = df['postStory'].apply(
    lambda x: (x.lower().count('shared ') if 'post' in x.lower() else 0) if isinstance(x, str) else 0
)

df['quantTimelinePhoto'] = df['postStory'].apply(
    lambda x: x.lower().count('shared a photo to your timeline.') if isinstance(x, str) else 0
)

df['quantTimelineVideo'] = df['postStory'].apply(
    lambda x: x.lower().count('shared a video to your timeline.') if isinstance(x, str) else 0
)

df['quantTimelineWrote'] = df['postStory'].apply(
    lambda x: x.lower().count('wrote on your timeline.') if isinstance(x, str) else 0
)

df['quantTimelineLink'] = df['postStory'].apply(
    lambda x: x.lower().count('a link to your timeline.') if isinstance(x, str) else 0
)

In [28]:
df['postCreated_time'] = pd.to_datetime(df['postCreated_time'])
df.head()

,_id,idade,sexo,id_usuario,nivel,pessimismo,tristeza,fracasso,prazer,culpa,...,quantAddPhotoWith,quantAddPhoto,quantSharedPhoto,quantSharedVideo,quantSharedLink,quantSharedPost,quantTimelinePhoto,quantTimelineVideo,quantTimelineWrote,quantTimelineLink
0,59305142c46708001d041a16,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,0,0,0,0,0,0,0,0
1,59305142c46708001d041a17,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,0,0,0,0,0,0,0,0
2,59305142c46708001d041a18,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,0,1,0,0,0,0,0,0
3,59305142c46708001d041a19,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,0,1,0,0,0,0,0,0
4,59305142c46708001d041a1a,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,0,1,0,0,0,0,0,0


In [29]:
df['data'] = df['postCreated_time'].dt.to_period('M').dt.to_timestamp()
df.head()

,_id,idade,sexo,id_usuario,nivel,pessimismo,tristeza,fracasso,prazer,culpa,...,quantAddPhoto,quantSharedPhoto,quantSharedVideo,quantSharedLink,quantSharedPost,quantTimelinePhoto,quantTimelineVideo,quantTimelineWrote,quantTimelineLink,data
0,59305142c46708001d041a16,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,0,0,0,0,0,0,0,2017-05-01
1,59305142c46708001d041a17,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,0,0,0,0,0,0,0,2017-05-01
2,59305142c46708001d041a18,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,1,0,0,0,0,0,0,2017-05-01
3,59305142c46708001d041a19,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,1,0,0,0,0,0,0,2017-04-01
4,59305142c46708001d041a1a,47,M,1402702123120502,2.0,1,3,3,3,1,...,0,0,1,0,0,0,0,0,0,2017-04-01


In [30]:
df_grouped_data = df.groupby(['id_usuario', 'data']).size().reset_index(name='quantidade')

In [31]:
for usuario_id in df_grouped_data['id_usuario'].unique():
    df_usuario = df_grouped_data[df_grouped_data['id_usuario'] == usuario_id]
    
    df_usuario.set_index('data', inplace=True)

    # Usar AutoregressionAD para detecção de anomalias
    autoreg_ad = AutoregressionAD(n_steps=1, step_size=1, c=3.0)
    anomalies = autoreg_ad.fit_detect()

    # Passo 4: Plotar os resultados
    plt.figure(figsize=(14, 7))

    # Plotar a série temporal
    sns.lineplot(x=s.index.astype(str), y=s.values, label='Quantidade de Posts')

    # Plotar as anomalias
    anomaly_points = s[anomalies == True]
    sns.scatterplot(x=anomaly_points.index.astype(str), y=anomaly_points.values, color='red', s=100, label='Anomalias')

    # Configurar o gráfico
    plt.title(f'Usuário {usuario_id} - Quantidade de Posts por Mês/Ano')
    plt.xlabel('Mês/Ano')
    plt.ylabel('Quantidade de Posts')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()

    # Mostrar o gráfico
    plt.show()

TypeError: _TrainableUnivariateDetector.fit_predict() missing 1 required positional argument: 'ts'